In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf
using JLD2

  Activating environment at `~/ocrl/ocrl_hw1/Project.toml`


## Q2 (20 pts): Augmented Lagrangian Quadratic Program Solver

Here we are going to use the augmented lagrangian method described [here in a video](https://www.youtube.com/watch?v=0x0JD5uO_ZQ), with [the corresponding pdf here](https://github.com/Optimal-Control-16-745/lecture-notebooks-2022/blob/main/misc/AL_tutorial.pdf) to solve the following problem:

$$\begin{align}
\min_x \quad & \frac{1}{2}x^TQx + q^Tx \\ 
\mbox{s.t.}\quad &  Ax -b = 0 \\ 
&  Gx - h \leq 0 
\end{align}$$
where the cost function is described by $Q \in \mathbb{R}^{n \times n}$, $q \in \mathbb{R}^n$, an equality constraint is described by $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$, and an inequality constraint is described by $G \in \mathbb{R}^{p \times n}$ and $h \in \mathbb{R}^p$.


By introducing a dual variable $\lambda \in \mathbb{R}^m$ for the equality constraint, and $\mu \in \mathbb{R}^p$ for the inequality constraint, we have the following KKT conditions for optimality:

$$\begin{align}
Qx + q + A^T\lambda + G^T \mu &= 0 \quad \quad \text{stationarity}\\ 
Ax-b&= 0 \quad \quad \text{primal feasibility} \\ 
Gx-h&\leq 0 \quad \quad \text{primal feasibility} \\ 
\mu &\geq 0 \quad \quad \text{dual feasibility} \\ 
\mu \circ (Gx - h) &= 0 \quad \quad \text{complementarity}
  \end{align}$$
  where $\circ$ is element-wise multiplication.  

In [21]:
# TODO: read below
# NOTE: DO NOT USE A WHILE LOOP ANYWHERE
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following function 
    
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)

You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:

as long as solve_qp works. 
"""
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end
function kkt_conditions(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector)::Vector
    h = h_ineq(qp, x)
   vcat(qp.Q*x + qp.q + qp.A'*λ + qp.G'*μ, c_eq(qp, x), max.(h, 0), min.(μ, 0), μ.*h)
end

function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix
    n = length(μ)
    my_matrix = zeros(n,n)
    h = h_ineq(qp, x)
    for i = 1:n
        if !((h[i] < 0) && (μ[i] == 0.0))
            my_matrix[i,i] = ρ
        end
    end
    return my_matrix     
end
function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real, mask_matrix::Matrix)::Real
    c = c_eq(qp, x)
    h = h_ineq(qp, x)
    0.5*x'*qp.Q*x + qp.q'*x + λ'*c + μ'*h + 0.5*ρ*dot(c,c) + 0.5*h'*mask_matrix*h
end
function logging(qp::NamedTuple, main_iter::Int, AL_gradient::Vector, x::Vector, λ::Vector, μ::Vector, ρ::Real)
    # TODO: stationarity norm
    stationarity_norm = norm(qp.Q*x + qp.q + qp.A'*λ + qp.G'*μ)
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, stationarity_norm, norm(AL_gradient), maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
function solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)
    x = zeros(length(qp.q))
    λ = zeros(length(qp.b))
    μ = zeros(length(qp.h))
    ρ = 1e-1
    ϕ = 10
    
    if verbose
        @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
        @printf "----------------------------------------------------------------\n"
    end
    
    AL_gradient = kkt_conditions(qp, x, λ, μ)
    
    # TODO:
    for main_iter = 1:max_iters 
        if verbose
            logging(qp, main_iter, AL_gradient, x, λ, μ, ρ)
        end
        
        # TODO: convergence criteria based on tol 
        if norm(AL_gradient) < tol
            return x, λ, μ
        end
        
        
        for inner_iter = 1:max_iters
            mask = mask_matrix(qp, x, μ, ρ)
            b = FD.gradient(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ, mask), x)
            
            if norm(b) < tol
                break
            end
            
            A = FD.hessian(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ, mask), x)
            
            step = - A \ b
            
            for backtrack_iter = 1:max_iters
                if norm(FD.gradient(_x -> augmented_lagrangian(qp, _x, λ, μ, ρ, mask), x + step)) < norm(b)
                    break
                end
                step /= 2
            end
            
            x += step
        end
        
        λ += ρ*c_eq(qp, x)
        μ = max.(0, μ + ρ*h_ineq(qp,x))
        ρ *= ϕ
        
        AL_gradient = kkt_conditions(qp, x, λ, μ)
    end
    error("qp solver did not converge")
end
let 
    # example solving qp 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, tol = 1e-8)
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   2.98e+01   3.09e+01   4.38e+00   6.49e+00   0.00e+00  1e-01
  2   4.31e-15   4.51e+00   2.41e+00   2.76e+00   1.21e+00  1e+00
  3   3.97e-15   1.24e+00   4.60e-01   1.10e+00   3.10e-01  1e+01
  4   6.23e+00   6.24e+00   2.50e-02   2.99e-01   7.96e-03  1e+02
  5   5.52e-01   5.53e-01   6.85e-03   1.37e-02   7.95e-03  1e+03
  6   3.94e-12   2.47e-04   3.64e-05   1.62e-04   1.06e-04  1e+04
  7   4.43e-11   2.78e-08  -5.62e-09   2.06e-08   1.14e-08  1e+05
  8   2.47e-10   2.47e-10  -1.57e-13   3.92e-13   1.89e-13  1e+06


([-0.3262308057134024, 0.24943797997188893, -0.432267664405071, -1.417224697124135, -1.3994527400876546, 0.6099582408523684, -0.07312202122159459, 1.3031477521999635, 0.5389034791065498, -0.7225813651685945], [-0.12835195123750232, -2.83762416728944, -0.8320804500157184], [0.03635294270409872, 0.0, 0.0, 1.0594444951408832, 0.0])

## QP Solver test (10 pts)

In [22]:
# 10 points 
using Test 
@testset "qp solver" begin 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-6)
    
    @load joinpath(@__DIR__, "qp_solutions.jld2") qp_solutions
    @test norm(x - qp_solutions.x,Inf)<1e-3;
    @test norm(λ - qp_solutions.λ,Inf)<1e-3;
    @test norm(μ - qp_solutions.μ,Inf)<1e-3;
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   2.98e+01   3.09e+01   4.38e+00   6.49e+00   0.00e+00  1e-01
  2   4.31e-15   4.51e+00   2.41e+00   2.76e+00   1.21e+00  1e+00
  3   3.97e-15   1.24e+00   4.60e-01   1.10e+00   3.10e-01  1e+01
  4   6.23e+00   6.24e+00   2.50e-02   2.99e-01   7.96e-03  1e+02
  5   5.52e-01   5.53e-01   6.85e-03   1.37e-02   7.95e-03  1e+03
  6   3.94e-12   2.47e-04   3.64e-05   1.62e-04   1.06e-04  1e+04
  7   4.43e-11   2.78e-08  -5.62e-09   2.06e-08   1.14e-08  1e+05
Test Summary: | Pass  Total
qp solver     |    3      3


Test.DefaultTestSet("qp solver", Any[], 3, false, false)

# Simulating a Falling Brick with QPs
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \lambda \\ \text{ where } M = mI_{2\times 2}, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\lambda \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ \Delta t \cdot \begin{bmatrix} \frac{1}{m} J^T \lambda_{k+1} - g \\ v_{k+1} \end{bmatrix}$$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\lambda_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\lambda_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (a): QP formulation (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + [M (\Delta t \cdot g - v_k)]^Tv_{k+1} \\
    &\text{subject to} && -J(q_k + \Delta t \cdot v_{k+1}) \leq 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown.

**PUT ANSWER HERE:**

## Brick Simulation (5 pts)

In [23]:
function brick_simulation_qp(q, v; mass = 1, Δt = 0.01)
    
    # TODO: fill in the QP problem data for a simulation step 
    # fill in Q, q, G, h, but leave A, b the same 
    # this is because there are no equality constraints in this qp 
    M = Matrix{Float64}(mass*I(2))
    g = [0 ; 9.81]
    J = [0 1]
    
    qp = (
        Q = M, 
        q = M*(Δt*g - v),
        A = zeros(0,2), # don't edit this
        b = zeros(0),   # don't edit this 
        G = -J*Δt,
        h = -J*q
    )
    
    return qp 
end

brick_simulation_qp (generic function with 1 method)

In [25]:
@testset "brick qp" begin 
    
    q = randn(2)
    v = randn(2)
    
    qp = brick_simulation_qp(q,v)
    
    # check all the types to make sure they're right
    qp.Q::Matrix{Float64}
    qp.q::Vector{Float64}
    qp.A::Matrix{Float64}
    qp.b::Vector{Float64}
    qp.G::Matrix{Float64}
    qp.h::Vector{Float64}
    
    @test size(qp.Q) == (2,2)
    @test size(qp.q) == (2,)
    @test size(qp.A) == (0,2)
    @test size(qp.b) == (0,)
    @test size(qp.G) == (1,2)
    @test size(qp.h) == (1,)
    display(qp.q)
    
    @test abs(tr(qp.Q) - 2) < 1e-10
    @test norm(qp.q - [-2.0, 3.0981]) < 1e-10 
    @test norm(qp.G - [0 -.01]) < 1e-10 
    @test abs(qp.h[1] -3) < 1e-10
    
end

2-element Vector{Float64}:
 -1.503056719171801
 -0.09950289449115744

Test Summary: | Pass  Total
brick qp      |    8      8


Test.DefaultTestSet("brick qp", Any[], 8, false, false)

In [28]:
include(joinpath(@__DIR__, "animate_brick.jl"))
let 
    
    dt = 0.01 
    T = 3.0 
    
    t_vec = 0:dt:T
    N = length(t_vec)
    
    qs = [zeros(2) for i = 1:N]
    vs = [zeros(2) for i = 1:N]
    
    qs[1] = [0, 1.0]
    vs[1] = [1, 4.5]
    
    # TODO: simulate the brick by forming and solving a qp 
    # at each timestep. Your QP should solve for vs[k+1], and
    # you should use this to update qs[k+1]
    
    for k = 1:(N-1)
        qp = brick_simulation_qp(qs[k], vs[k]; Δt=dt)
        vs[k+1], λ, μ = solve_qp(qp)
        qs[k+1] = qs[k] + dt*vs[k+1]
    end

    
    xs = [q[1] for q in qs]
    ys = [q[2] for q in qs]
    
    @show @test abs(maximum(ys)-2)<1e-1
    @show @test minimum(ys) > -1e-2
    @show @test abs(xs[end] - 3) < 1e-2
    
    xdot = diff(xs)/dt
    @show @test maximum(xdot) < 1.0001
    @show @test minimum(xdot) > 0.9999
    @show @test ys[110] > 1e-2
    @show @test abs(ys[111]) < 1e-2
    @show @test abs(ys[112]) < 1e-2
    
    display(plot(xs, ys, ylabel = "y (m)", xlabel = "x (m)"))
    
    animate_brick(qs)
    
    
    
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   4.51e+00   4.62e+00   1.00e+00   0.00e+00   0.00e+00  1e-01
  2   2.11e-16   9.60e-01   9.56e-01   0.00e+00   9.14e-02  1e+00
  3   1.02e-16   1.39e+00   9.56e-01   0.00e+00   1.01e+00  1e+01
  4   5.55e-17   1.02e+01   9.55e-01   0.00e+00   1.01e+01  1e+02
  5   2.22e-16   9.94e+01   9.45e-01   0.00e+00   9.94e+01  1e+03
  6   0.00e+00   8.29e+02   8.60e-01   0.00e+00   8.29e+02  1e+04
  7   7.11e-15   2.26e+03   4.30e-01   0.00e+00   2.26e+03  1e+05
  8   4.26e-14   3.58e+02   3.91e-02   0.00e+00   3.58e+02  1e+06
  9   3.13e-13   3.70e+00   3.87e-04   0.00e+00   3.70e+00  1e+07
 10   1.09e-12   3.69e-03   3.86e-07   0.00e+00   3.69e-03  1e+08
 11   8.51e-12   3.69e-07   3.86e-11   0.00e+00   3.69e-07  1e+09
 12   5.86e-10   5.86e-10   4.44e-16   0.00e+00   4.25e-12  1e+10
iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
-----------------

LoadError: qp solver did not converge